In [ ]:
# One-liner: Import libraries, set options, and lock a random seed for reproducibility.
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
np.random.seed(42)
plt.rcParams["figure.figsize"] = (8,5)
